# Code Review
이 코드는 https://wikidocs.net/21667 사이트의 코드를 가져와 리뷰한 것입니다.

In [1]:
# 카운트 기반의 단어 표현(BoW)
from konlpy.tag import Okt
import re  
okt=Okt()  

token=re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")  
# 정규 표현식을 통해 온점을 제거하는 정제 작업입니다.  
token=okt.morphs(token)  
# OKT 형태소 분석기를 통해 토큰화 작업을 수행한 뒤에, token에다가 넣습니다.  

word2index={}  
bow=[]  
for voca in token:  
         if voca not in word2index.keys():  
             word2index[voca]=len(word2index)  
# token을 읽으면서, word2index에 없는 (not in) 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.   
             bow.insert(len(word2index)-1,1)
# BoW 전체에 전부 기본값 1을 넣어줍니다. 단어의 개수는 최소 1개 이상이기 때문입니다.  
         else:
            index=word2index.get(voca)
# 재등장하는 단어의 인덱스를 받아옵니다.
            bow[index]=bow[index]+1
# 재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다. (단어의 개수를 세는 것입니다.)  
print(word2index)

C:\anaconda\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


In [3]:
print(token)
print(bow)

['정부', '가', '발표', '하는', '물가상승률', '과', '소비자', '가', '느끼는', '물가상승률', '은', '다르다']
[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [5]:
# 직접 불용어 입력하기
text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [6]:
# Tensor의 Embedding 활용하기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [8]:
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(t.word_index)

{'nice': 1, 'great': 2, 'best': 3, 'amazing': 4, 'stop': 5, 'lies': 6, 'pitiful': 7, 'nerd': 8, 'excellent': 9, 'work': 10, 'supreme': 11, 'quality': 12, 'bad': 13, 'highly': 14, 'respectable': 15}


In [11]:
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

max_len=max(len(l) for l in X_encoded)
print(max_len)

X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]
4
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len)) # 모든 임베딩 벡터는 4차원.
model.add(Flatten()) # Dense의 입력으로 넣기위함.
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Train on 7 samples
Epoch 1/100
7/7 - 5s - loss: 0.6879 - acc: 0.7143
Epoch 2/100
7/7 - 0s - loss: 0.6866 - acc: 0.7143
Epoch 3/100
7/7 - 0s - loss: 0.6852 - acc: 0.8571
Epoch 4/100
7/7 - 0s - loss: 0.6839 - acc: 0.8571
Epoch 5/100
7/7 - 0s - loss: 0.6825 - acc: 0.8571
Epoch 6/100
7/7 - 0s - loss: 0.6812 - acc: 0.8571
Epoch 7/100
7/7 - 0s - loss: 0.6798 - acc: 0.8571
Epoch 8/100
7/7 - 0s - loss: 0.6785 - acc: 0.8571
Epoch 9/100
7/7 - 0s - loss: 0.6771 - acc: 0.8571
Epoch 10/100
7/7 - 0s - loss: 0.6758 - acc: 0.8571
Epoch 11/100
7/7 - 0s - loss: 0.6744 - acc: 0.8571
Epoch 12/100
7/7 - 0s - loss: 0.6730 - acc: 0.8571
Epoch 13/100
7/7 - 0s - loss: 0.6717 - acc: 0.8571
Epoch 14/100
7/7 - 0s - loss: 0.6703 - acc: 1.0000
Epoch 15/100
7/7 - 0s - loss: 0.6689 - acc: 1.0000
Epoch 16/100
7/7 - 0s - loss: 0.6676 - acc: 1.0000
Epoch 17/100
7/7 - 0s - loss: 0.6662 - acc: 1.0000
Epoch 18/100
7/7 - 0s - loss: 0.6648 - acc: 1.0000
Epoch 19/100
7/7 - 0s - loss: 0.6634 - acc: 1.0000
Epoch 20/100
7/7 - 0s

In [16]:
# glove 활용 embedding
n=0
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
    print(word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫번째 값만 저장
    print(word) # word_vector의 첫번째 값만 출력
    n=n+1
    if n==2:
        break
f.close()

['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.538', '-0.30223

In [17]:
import numpy as np
embedding_dict = dict()
f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [23]:
print(embedding_dict['nice'])
print(len(embedding_dict['respectable']))

[-0.18554    0.047152   0.34867   -0.23114   -0.26083    0.63108
  0.55687    0.61622   -0.15437   -0.38381    0.12445   -0.26999
 -0.29196    0.1125     0.36035    0.70689   -0.33891   -0.2695
  0.17481    0.97048    0.23014    0.63168   -0.24543   -0.7289
  0.32517   -0.21118   -0.80354   -0.59863   -0.10182   -0.87826
 -0.80162    0.20999    0.64598   -0.38239    0.64512    0.73046
 -0.19881    0.35717    0.19135   -0.43686    0.75956   -0.66431
  0.34509   -1.0383    -0.50491    0.19976   -0.041208   0.16952
  0.17821   -0.84249    0.21992   -0.16474   -0.24669    0.34117
 -0.59713   -2.3434     0.31484    0.69668    0.53368   -0.62857
 -0.197      0.52241   -1.5903    -0.16475    0.62553   -0.094116
  0.0070705  0.22617   -0.45698   -0.53268    0.11573   -0.19052
  0.28086   -0.5591     0.35291    0.42552    0.47334   -0.41046
  0.47275    0.22753   -0.097073  -0.11809   -0.25613   -0.10689
 -1.2788    -0.085504   0.20205    0.23338   -0.52939   -0.49492
  0.15289   -0.31324    0.

In [19]:
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(16, 100)

In [20]:
print(t.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [24]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다
    temp = embedding_dict.get(word) # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

In [26]:
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Train on 7 samples
Epoch 1/100
7/7 - 2s - loss: 0.6841 - acc: 0.5714
Epoch 2/100
7/7 - 0s - loss: 0.6670 - acc: 0.7143
Epoch 3/100
7/7 - 0s - loss: 0.6504 - acc: 0.7143
Epoch 4/100
7/7 - 0s - loss: 0.6342 - acc: 0.7143
Epoch 5/100
7/7 - 0s - loss: 0.6185 - acc: 0.7143
Epoch 6/100
7/7 - 0s - loss: 0.6033 - acc: 0.7143
Epoch 7/100
7/7 - 0s - loss: 0.5885 - acc: 0.7143
Epoch 8/100
7/7 - 0s - loss: 0.5741 - acc: 0.7143
Epoch 9/100
7/7 - 0s - loss: 0.5601 - acc: 0.7143
Epoch 10/100
7/7 - 0s - loss: 0.5465 - acc: 0.7143
Epoch 11/100
7/7 - 0s - loss: 0.5333 - acc: 0.7143
Epoch 12/100
7/7 - 0s - loss: 0.5205 - acc: 0.7143
Epoch 13/100
7/7 - 0s - loss: 0.5080 - acc: 0.7143
Epoch 14/100
7/7 - 0s - loss: 0.4959 - acc: 0.7143
Epoch 15/100
7/7 - 0s - loss: 0.4841 - acc: 0.7143
Epoch 16/100
7/7 - 0s - loss: 0.4726 - acc: 0.7143
Epoch 17/100
7/7 - 0s - loss: 0.4614 - acc: 0.7143
Epoch 18/100
7/7 - 0s - loss: 0.4506 - acc: 0.7143
Epoch 19/100
7/7 - 0s - loss: 0.4400 - acc: 0.7143
Epoch 20/100
7/7 - 0s